In [1]:
# all imports
import pandas as pd
import numpy as np
import pyspark
from pydataset import data
from pyspark.sql.functions import sum, mean, concat, lit, regexp_extract, regexp_replace, when
from vega_datasets import data
from pyspark.sql.functions import month, year, quarter
from pyspark.sql.functions import *

### Create a spark data frame that contains your favorite programming languages.

- The name of the column should be language

- View the schema of the dataframe

- Output the shape of the dataframe

- Show the first 5 records in the dataframe

In [2]:
import pyspark

spark = pyspark.sql.SparkSession.builder.getOrCreate()

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/05/18 11:02:01 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
import pandas as pd
import numpy as np

np.random.seed(456)

# Create a spark data frame that contains your favorite programming languages.
lang_df = pd.DataFrame({"language": ["Python", "Scala", "R", "Java", "C++", "Haskell", "Clojure"]})
lang_df.head()

,language
0,Python
1,Scala
2,R
3,Java
4,C++


In [4]:
df = spark.createDataFrame(lang_df)

In [5]:
# View the schema of the dataframe
df.printSchema()

root
 |-- language: string (nullable = true)



In [6]:
# Output the shape of the dataframe
df.count()

7

In [7]:
# Show the first 5 records in the dataframe
df.show(5)

+--------+
|language|
+--------+
|  Python|
|   Scala|
|       R|
|    Java|
|     C++|
+--------+
only showing top 5 rows



### Load the mpg dataset as a spark dataframe.

Create 1 column of output that contains a message like the one below:


The 1999 audi a4 has a 4 cylinder engine.
For each vehicle.

Transform the trans column so that it only contains either manual or auto.

In [8]:
# Load the mpg dataset as a spark dataframe.
from pydataset import data

mpg = spark.createDataFrame(data("mpg"))
mpg.show(5)
# Load the mpg dataset as a spark dataframe.

+------------+-----+-----+----+---+----------+---+---+---+---+-------+
|manufacturer|model|displ|year|cyl|     trans|drv|cty|hwy| fl|  class|
+------------+-----+-----+----+---+----------+---+---+---+---+-------+
|        audi|   a4|  1.8|1999|  4|  auto(l5)|  f| 18| 29|  p|compact|
|        audi|   a4|  1.8|1999|  4|manual(m5)|  f| 21| 29|  p|compact|
|        audi|   a4|  2.0|2008|  4|manual(m6)|  f| 20| 31|  p|compact|
|        audi|   a4|  2.0|2008|  4|  auto(av)|  f| 21| 30|  p|compact|
|        audi|   a4|  2.8|1999|  6|  auto(l5)|  f| 16| 26|  p|compact|
+------------+-----+-----+----+---+----------+---+---+---+---+-------+
only showing top 5 rows



In [9]:
# Create 1 column of output that contains a message like the one below:
# 'The 1999 audi a4 has a 4 cylinder engine.'
# For each vehicle once.

mpg.select(
    concat(
        lit("The "),
        col("year"),
        lit(" "),
        col("manufacturer"),
        lit(" "),
        col("model"),
        lit(" has a "),
        col("cyl"),
        lit(" cylinder engine."),
    ).alias("vehicle_cylinder_desc")
).show(truncate=False)


+--------------------------------------------------------------+
|vehicle_cylinder_desc                                         |
+--------------------------------------------------------------+
|The 1999 audi a4 has a 4 cylinder engine.                     |
|The 1999 audi a4 has a 4 cylinder engine.                     |
|The 2008 audi a4 has a 4 cylinder engine.                     |
|The 2008 audi a4 has a 4 cylinder engine.                     |
|The 1999 audi a4 has a 6 cylinder engine.                     |
|The 1999 audi a4 has a 6 cylinder engine.                     |
|The 2008 audi a4 has a 6 cylinder engine.                     |
|The 1999 audi a4 quattro has a 4 cylinder engine.             |
|The 1999 audi a4 quattro has a 4 cylinder engine.             |
|The 2008 audi a4 quattro has a 4 cylinder engine.             |
|The 2008 audi a4 quattro has a 4 cylinder engine.             |
|The 1999 audi a4 quattro has a 6 cylinder engine.             |
|The 1999 audi a4 quattro

Transform the trans column so that it only contains either manual or auto.

In [10]:
# Transform the trans column so that it only contains either manual or auto.
mpg = mpg.select("*", mpg.trans, regexp_extract("trans", r"(.\w+)", 1).alias('transformed_trans'))
mpg.show(5)

+------------+-----+-----+----+---+----------+---+---+---+---+-------+----------+-----------------+
|manufacturer|model|displ|year|cyl|     trans|drv|cty|hwy| fl|  class|     trans|transformed_trans|
+------------+-----+-----+----+---+----------+---+---+---+---+-------+----------+-----------------+
|        audi|   a4|  1.8|1999|  4|  auto(l5)|  f| 18| 29|  p|compact|  auto(l5)|             auto|
|        audi|   a4|  1.8|1999|  4|manual(m5)|  f| 21| 29|  p|compact|manual(m5)|           manual|
|        audi|   a4|  2.0|2008|  4|manual(m6)|  f| 20| 31|  p|compact|manual(m6)|           manual|
|        audi|   a4|  2.0|2008|  4|  auto(av)|  f| 21| 30|  p|compact|  auto(av)|             auto|
|        audi|   a4|  2.8|1999|  6|  auto(l5)|  f| 16| 26|  p|compact|  auto(l5)|             auto|
+------------+-----+-----+----+---+----------+---+---+---+---+-------+----------+-----------------+
only showing top 5 rows



### Load the tips dataset as a spark dataframe.

What percentage of observations are smokers?

Create a column that contains the tip percentage
Calculate the average tip percentage for each combination of sex and smoker.

In [11]:
# load the tips dataset
tips = spark.createDataFrame(data('tips'))

In [12]:
# view data
tips.show(3)

+----------+----+------+------+---+------+----+
|total_bill| tip|   sex|smoker|day|  time|size|
+----------+----+------+------+---+------+----+
|     16.99|1.01|Female|    No|Sun|Dinner|   2|
|     10.34|1.66|  Male|    No|Sun|Dinner|   3|
|     21.01| 3.5|  Male|    No|Sun|Dinner|   3|
+----------+----+------+------+---+------+----+
only showing top 3 rows



What percentage of observations are smokers?

In [13]:
tips.createOrReplaceTempView("tips")

In [14]:
# run SQL query using spark
spark.sql(
    """
SELECT ((SELECT COUNT(smoker)
FROM tips
WHERE smoker = 'Yes') / 
(SELECT COUNT(smoker)
FROM tips)*100) as pct_smokers
FROM tips
"""
).show(1)

+------------------+
|       pct_smokers|
+------------------+
|38.114754098360656|
+------------------+
only showing top 1 row



Create a column that contains the tip percentage

In [15]:
# create column that contains the tip percentage
# calculate tip percent
col = (tips.tip / tips.total_bill) * 100
# create new column
tip_df = tips.select('*', col.alias('tip_per'))

In [16]:
tip_df.show(3)

+----------+----+------+------+---+------+----+------------------+
|total_bill| tip|   sex|smoker|day|  time|size|           tip_per|
+----------+----+------+------+---+------+----+------------------+
|     16.99|1.01|Female|    No|Sun|Dinner|   2|5.9446733372572105|
|     10.34|1.66|  Male|    No|Sun|Dinner|   3|16.054158607350097|
|     21.01| 3.5|  Male|    No|Sun|Dinner|   3|16.658733936220845|
+----------+----+------+------+---+------+----+------------------+
only showing top 3 rows



Calculate the average tip percentage for each combination of sex and smoker.

In [17]:
# Use groupby to find the average tip for sex and smoker
tip_df.groupby("sex").pivot('smoker').mean('tip_per').show()

+------+------------------+------------------+
|   sex|                No|               Yes|
+------+------------------+------------------+
|Female|15.692097076918358| 18.21503526994103|
|  Male| 16.06687151291298|15.277117520248513|
+------+------------------+------------------+



Use the seattle weather dataset referenced in the lesson to answer the questions below.

- Convert the temperatures to fahrenheit.

- Which month has the most rain, on average?

- Which year was the windiest?

- What is the most frequent type of weather in January?

- What is the average high and low temperature on sunny days in July in 2013 and 2014?

- What percentage of days were rainy in q3 of 2015?

- For each year, find what percentage of days it rained (had non-zero precipitation).

In [18]:
# acuire data from vega_datasets

from vega_datasets import data

weather = data.seattle_weather().assign(date=lambda df: df.date.astype(str))
weather = spark.createDataFrame(weather)
weather.show(6)

+----------+-------------+--------+--------+----+-------+
|      date|precipitation|temp_max|temp_min|wind|weather|
+----------+-------------+--------+--------+----+-------+
|2012-01-01|          0.0|    12.8|     5.0| 4.7|drizzle|
|2012-01-02|         10.9|    10.6|     2.8| 4.5|   rain|
|2012-01-03|          0.8|    11.7|     7.2| 2.3|   rain|
|2012-01-04|         20.3|    12.2|     5.6| 4.7|   rain|
|2012-01-05|          1.3|     8.9|     2.8| 6.1|   rain|
|2012-01-06|          2.5|     4.4|     2.2| 2.2|   rain|
+----------+-------------+--------+--------+----+-------+
only showing top 6 rows



In [19]:
# Convert the temperatures to fahrenheit.
weather = weather.withColumn('temp_max_F', ((weather.temp_max.cast('double') * 9 / 5) + 32)).drop('temp_max')
weather = weather.withColumn('temp_min_F', ((weather.temp_min.cast('double') * 9 / 5) + 32)).drop('temp_min')
weather.show(5)

+----------+-------------+----+-------+----------+----------+
|      date|precipitation|wind|weather|temp_max_F|temp_min_F|
+----------+-------------+----+-------+----------+----------+
|2012-01-01|          0.0| 4.7|drizzle|     55.04|      41.0|
|2012-01-02|         10.9| 4.5|   rain|     51.08|     37.04|
|2012-01-03|          0.8| 2.3|   rain|     53.06|     44.96|
|2012-01-04|         20.3| 4.7|   rain|     53.96|     42.08|
|2012-01-05|          1.3| 6.1|   rain|     48.02|     37.04|
+----------+-------------+----+-------+----------+----------+
only showing top 5 rows



In [20]:
# Which month has the most rain, on average?
(weather.withColumn('month', month('date'))
    .groupBy('month')
    .agg(mean('precipitation').alias('avg_rain'))
    .sort(desc('avg_rain'))
    .show(1))

+-----+-----------------+
|month|         avg_rain|
+-----+-----------------+
|   11|5.354166666666667|
+-----+-----------------+
only showing top 1 row



In [21]:
# Which year was the windiest?
(
    weather.withColumn("year", year("date"))
    .groupBy("year")
    .agg(sum("wind").alias("total_wind"))
    .sort(desc("total_wind"))
    .head(5)
)

[Row(year=2012, total_wind=1244.6999999999998),
 Row(year=2014, total_wind=1236.5000000000005),
 Row(year=2015, total_wind=1153.3),
 Row(year=2013, total_wind=1100.8000000000002)]

In [29]:
# What is the most frequent type of weather in January?
(
    weather.withColumn("month", month("date"))
    .filter(column("month") == 1)
    .groupBy("weather")
    .count()
    .sort(desc("count"))
    .show()
)

+-------+-----+
|weather|count|
+-------+-----+
|    fog|   38|
|   rain|   35|
|    sun|   33|
|drizzle|   10|
|   snow|    8|
+-------+-----+



In [31]:
# What is the average high and low temperature on sunny days in July in 2013 and 2014?
(
    weather.filter(month("date") == 7)
    .filter(year("date") > 2012)
    .filter(year("date") < 2015)
    .filter(column("weather") == lit("sun"))
    .agg(
        avg("temp_max_F").alias("average_high_temp"),
        avg("temp_min_F").alias("average_low_temp"),
    )
    .show()
)

+-----------------+-----------------+
|average_high_temp| average_low_temp|
+-----------------+-----------------+
|80.29192307692308|57.52884615384615|
+-----------------+-----------------+



In [34]:
# What percentage of days were rainy in q3 of 2015?
(
    weather.filter(year("date") == 2015)
    .filter(quarter("date") == 3)
    .select(when(column("weather") == "rain", 1).otherwise(0).alias("rain"))
    .agg(mean("rain")*100)
    .show()
)

+------------------+
| (avg(rain) * 100)|
+------------------+
|2.1739130434782608|
+------------------+



In [36]:
# For each year, find what percentage of days it rained (had non-zero precipitation).
(
    weather.withColumn("year", year("date"))
    .select(when(column("precipitation") > 0, 1).otherwise(0).alias("rain"), "year")
    .groupby("year")
    .agg(mean("rain"))
    .show()
)

+----+-------------------+
|year|          avg(rain)|
+----+-------------------+
|2012|0.48360655737704916|
|2013|0.41643835616438357|
|2014|  0.410958904109589|
|2015|0.39452054794520547|
+----+-------------------+

